In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# ======================================================
# CONFIGURATION
# ======================================================

INPUT_FILE = Path(r"D:\BREACH CLASSIFIER\breaches_classified_3.xlsx")
OUTPUT_DIR = Path("plots_output")
OUTPUT_DIR.mkdir(exist_ok=True)

DPI = 400
MIN_PERCENT_THRESHOLD = 2  # Group small slices

# ======================================================
# LOAD ALL SHEETS
# ======================================================

excel_data = pd.read_excel(INPUT_FILE, sheet_name=None)

print(f"Loaded {len(excel_data)} sheets.")

# ======================================================
# GLOBAL COLOR MAP (CONSISTENT ACROSS ALL SHEETS)
# ======================================================

all_categories = set()

for df in excel_data.values():
    if "Attack Subcategory (Hybrid)" in df.columns:
        all_categories.update(df["Attack Subcategory (Hybrid)"].dropna().unique())
    if "Attack Subcategory (LLM)" in df.columns:
        all_categories.update(df["Attack Subcategory (LLM)"].dropna().unique())

all_categories = sorted(all_categories)

base_colors = list(mcolors.TABLEAU_COLORS.values())
color_map = {
    cat: base_colors[i % len(base_colors)]
    for i, cat in enumerate(all_categories)
}

# ======================================================
# CLEAN PIE FUNCTION
# ======================================================

def create_clean_pie(values, categories, title, output_prefix):

    total = sum(values)
    percentages = [(v / total) * 100 if total > 0 else 0 for v in values]

    new_values = []
    new_categories = []
    new_colors = []

    minor_total = 0

    for v, p, cat in zip(values, percentages, categories):
        if p < MIN_PERCENT_THRESHOLD:
            minor_total += v
        else:
            new_values.append(v)
            new_categories.append(cat)
            new_colors.append(color_map[cat])

    if minor_total > 0:
        new_values.append(minor_total)
        new_categories.append(f"Minor Categories (<{MIN_PERCENT_THRESHOLD}%)")
        new_colors.append("#cccccc")

    # ---------- PIE FIGURE ----------
    fig, ax = plt.subplots(figsize=(12, 12), constrained_layout=True)

    wedges, texts, autotexts = ax.pie(
        new_values,
        colors=new_colors,
        autopct=lambda pct: f"{pct:.1f}%" if pct >= MIN_PERCENT_THRESHOLD else "",
        startangle=140,
        wedgeprops=dict(edgecolor="white"),
        textprops=dict(fontsize=12)
    )

    ax.set_title(title, fontsize=16)

    plt.savefig(f"{output_prefix}_Pie.png", dpi=DPI)
    plt.close()

    # ---------- LEGEND FIGURE ----------
    fig, ax = plt.subplots(figsize=(14, 4))
    ax.axis("off")

    legend_handles = [
        plt.Line2D([0], [0], marker='o', color='w',
                   markerfacecolor=col, markersize=12)
        for col in new_colors
    ]

    ax.legend(
        legend_handles,
        new_categories,
        loc="center",
        ncol=3,
        frameon=False,
        fontsize=11
    )

    plt.savefig(f"{output_prefix}_Legend.png", dpi=DPI)
    plt.close()


# ======================================================
# PROCESS EACH SHEET
# ======================================================

for sheet_name, df in excel_data.items():

    print(f"Processing sheet: {sheet_name}")

    required_cols = {
        "Attack Subcategory (Hybrid)",
        "Attack Subcategory (LLM)",
        "Trust Score (%)",
        "Trust Score (LLM)"
    }

    if not required_cols.issubset(df.columns):
        print(f"Skipping {sheet_name} (missing required columns)")
        continue

    # --------------------------------------------------
    # Distribution Counts
    # --------------------------------------------------

    hybrid_counts = df["Attack Subcategory (Hybrid)"].value_counts()
    llm_counts = df["Attack Subcategory (LLM)"].value_counts()

    categories = sorted(set(hybrid_counts.index).union(set(llm_counts.index)))

    hybrid_vals = [hybrid_counts.get(cat, 0) for cat in categories]
    llm_vals = [llm_counts.get(cat, 0) for cat in categories]

    # --------------------------------------------------
    # CLEAN PIE – HYBRID
    # --------------------------------------------------

    create_clean_pie(
        hybrid_vals,
        categories,
        f"{sheet_name} – Hybrid Distribution",
        OUTPUT_DIR / f"{sheet_name}_Hybrid"
    )

    # --------------------------------------------------
    # CLEAN PIE – LLM
    # --------------------------------------------------

    create_clean_pie(
        llm_vals,
        categories,
        f"{sheet_name} – LLM Distribution",
        OUTPUT_DIR / f"{sheet_name}_LLM"
    )

    # --------------------------------------------------
    # COMPARISON BAR CHART
    # --------------------------------------------------

    x = np.arange(len(categories))
    width = 0.35

    fig, ax = plt.subplots(figsize=(16, 8), constrained_layout=True)

    ax.bar(
        x - width/2,
        hybrid_vals,
        width,
        label="Hybrid",
        color=[color_map[c] for c in categories]
    )

    ax.bar(
        x + width/2,
        llm_vals,
        width,
        label="LLM",
        color=[color_map[c] for c in categories],
        alpha=0.6
    )

    ax.set_xticks(x)
    ax.set_xticklabels(categories, rotation=45, ha="right")
    ax.set_ylabel("Count")
    ax.set_title(f"{sheet_name} – Hybrid vs LLM Comparison")
    ax.legend()

    plt.savefig(OUTPUT_DIR / f"{sheet_name}_Comparison.png", dpi=DPI)
    plt.close()

print("All plots generated successfully.")

Loaded 19 sheets.
Processing sheet: network server
Processing sheet: email
Processing sheet: electronic medical record, net
Processing sheet: electronic medical record
Processing sheet: laptop
Processing sheet: desktop computer
Processing sheet: desktop computer, network serv
Processing sheet: desktop computer, electronic m
Processing sheet: email, network server
Processing sheet: electronic medical record, lap
Processing sheet: laptop, network server
Processing sheet: desktop computer, laptop
Processing sheet: desktop computer, email, netwo
Processing sheet: electronic medical record, ema
Processing sheet: desktop computer, email
Processing sheet: email, laptop
Processing sheet: desktop computer, email, lapto
Processing sheet: desktop computer, laptop, netw
Processing sheet: email, laptop, network server
All plots generated successfully.
